**Engy-5310** Computational Continuum Transport Phenomena, UMass Lowell, Chem. Eng. Nuclear Program Spring 2021; Prof. V. F. de Almeida

# Xenon-135 Removal from Molten Salt Reactors by Helium Gas Bubbles
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Student(s):** Anadi Mondal <br>
**Course:** Engy-5310 Computational Continuum Transport Phenomena  <br>
**Mentor:**Prof. Subash L. Sharma 

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**27Mar2021**

---
## Table of Contents<a id="toc"></a>
* [Introduction](#intro)
* [Problem Statement](#problem)
* [Objectives](#obj)
* [Approach](#app)
 + [Weak Form of Problem Statement](#weakform)
 + [MOOSE Kernels](#kernels)
 + [Setup](#setup)
 + [Tests](#tests)
   - [1-D](#1d-test)
   - [2-D](#1d-test)
* [Results and Analysis](#ranalysis)
* [Conclusions](#conclusions)
* [References](#refs)
---

## [Introduction](#toc)<a id="intro"></a>

- Xenon-135 buildup in the molten salt reactor is dangerous for the sustaining running of reactor. As xenon has very high absorption cross section, it absorb neutron which hinders the continuous chain reaction for required power production. So, removing of xenon form the reactor core is must. One of the ways to remove xenon is to sparge helium gas as bubble in the reactor core because xenon is easily absorbed in helium bubble. The amount of xenon transfer to the helium is highly dependent on helium void fraction. If we increase the void fraction of helium in the system it will increase the interfacial area which is responsible for transferring xenon from salt to helium bubble.

## [Problem Statement](#toc)<a id="problem"></a>

- The Xe-135 is stripped from the Molten salt by interfacial Mass transfer process using the He Gas. The general transport equation for this is

So the first step is in this direction is to simplify the problem. 

- The strong form of the two coupled equations are written as,

The mass conservation equation of xenon in salt,

\begin{align*}
( 1- \alpha)\times v\, u_1' &= -\bigl(-D_1\,( 1-\alpha)\times u_1'\bigr)'(x) - S(u_1, u_2) \quad \forall \quad x\in [a,b], \\
 u_1(a) &= A_1, \\
 u_1(b)' &= B_1.
\end{align*}

And the conservation equation of xenon in helium gas is as follows,


\begin{align*}
 \alpha\times v\, u_2' &= -\bigl(-D_2\,\alpha\times u_2'\bigr)'(x) + S(u_1, u_2) \quad \forall \quad x\in [a,b], \\
 u_2(a) &= A_2, \\
 u_2(b)' &= B_2.
\end{align*}





- Source coupling:

\begin{align*}
 S(u_1, u_2) = m \times a_i\bigl(u_1-H\,u_2\bigr)
\end{align*}

Where,m= mass transfer coefficient,a_i= interfacial area, H= Henry Constant

- Assumptions

1.Considering a homogeneous model, means salt and helium bubble are flowing at same speed.

2.Concentration change in one direction(1D problem)

3.Steady State Case

4.The system is dominated by convection ,so diffusion term is negligible, can be ignored


## [Objectives](#toc)<a id="obj"></a>

- To determine the concentration change of Xenon in fuel salt
- To determine the buildup of xenon in helium bubble


## [Approach](#toc)<a id="app"></a>

- The weak form of the two conservation equation can be written as,

The Galerkin weak formulation is as follows. Find $u_1 \in H^1\!\bigl([a,b]\bigr)$ and $u_2 \in H^1\!\bigl([a,b]\bigr)$
so that 

\begin{align*}
 \int\limits_a^b v\, u_1'(x)\, w(x)\,dx + \int\limits_a^b D_1\, u_1'(x)\,w'(x)\,dx + \int\limits_a^b S(u_1, u_2)\,w(x)\,dx &= 0 \quad \forall \quad w \in H^1_0\!\bigl([a,b]\bigr), \text{and}
 \\
  \int\limits_a^b v\, u_2'(x)\, w(x)\,dx + \int\limits_a^b D_2\, u_2'(x)\,w'(x)\,dx - \int\limits_a^b S(u_1, u_2)\,w(x)\,dx &= 0 \quad \forall \quad w \in H^1_0\!\bigl([a,b]\bigr),
\end{align*}

where $H^1\!\bigl([a,b]\bigr) := \bigl\{ u:[a,b]\subset\ Reals\rightarrow \ Reals \mid \int_a^b u'^2\,dx < \infty\bigr\}$ and $H^1_0\!\bigl([a,b]\bigr) := \bigl\{ w \mid w \in H^1(a,b), w(a) = 0, w(b)' =0 \bigr\}$. Both function sets as just defined are Hilbert spaces. The function $w$ is called a test function. Because $w$, $u_1$, $u_2$ are sought in very similar sets of functions, this weak form is called Galerkin's weak form.

 1. $\pm S(u_1, u_2)\,w(x)$.

- Need to use 3 MOOSE kernels
1. Diffusion Kernel
2. Convection Kernel
3. Source Kernel

Solve problem with parameter values:

> + $a = 0$ ft
> + $b = 6$ ft



| $u_1$ **Parameter** | **Value**  | $u_2$ **Parameter** | **Value** |
|:-------------------:|:----------:|:-------------------:|:---------:|
| $A_1$               | 0.0459 mole/ft3     |  $A_2$              | 0 mole/ft3    |
| $B_1$               | 0 mole/ft^2-hr     |  $B_2$              |  0 mole/ft^2-hr    |
| $D_1$               | $5\times 10^{-5}$ ft^2/hr |  $D_2$              | $9.2\times 10^{-5}$ ft^2/hr  |


FEM parameters:

> + Basis Functions: First Order Lagrangian
> + num. of finite elements: 20

In [ ]:

'''Plot function for FEM Solution'''

def plot_solution(df,
                  dimension='Null',
                  title='No Title', 
                  basis_functions_type='No basis functions type',
                  flux_basis_functions_type='No basis functions type'):
    
    import matplotlib.pyplot as plt
    %matplotlib inline
    import numpy as np
    plt.style.use('dark_background') 
    axial="x"
    axial_1="x1"
    axial_2="x2"
    (fig, ax1) = plt.subplots(1, figsize=(14, 5))
    
    #Plotting for concentration and flux of Xenon in Salt
    
    if (axial == dimension):
        ax1.plot(df['x'], df['u'],'r*-',label=basis_functions_type)

        ax1.set_xlabel(r'$x$ [ft]', fontsize=18)
        ax1.set_ylabel(r'$u_h(x)$ [mole/ft3]', fontsize=18, color='red')

        ax1.tick_params(axis='y', labelcolor='red', labelsize=14)
        ax1.tick_params(axis='x', labelsize=14)
        ax1.legend(loc='center left', fontsize=12)
        ax1.grid(True)
        
        ## Flux Plot 
            
        ax2 = ax1.twinx()

        ax2.plot(df['x'], df['diffFluxU_x'],'b*-',label=flux_basis_functions_type)
        
        ax2.set_ylabel(r"$q_h(x)$ [mole/ft2-hr]", fontsize=16, color='lightblue')
        ax2.tick_params(axis='y', labelcolor='lightblue', labelsize=14)
        ax2.legend(loc='center right', fontsize=12)
        
     #Plotting for concentration and flux of Xenon in Helium
    
    if (axial_1 == dimension):
        ax1.plot(df['x'], df['u2'],'r*-',label=basis_functions_type)

        ax1.set_xlabel(r'$x$ [ft]', fontsize=18)
        ax1.set_ylabel(r'$u_h(x)$ [mole/ft3]', fontsize=18, color='red')

        ax1.tick_params(axis='y', labelcolor='red', labelsize=14)
        ax1.tick_params(axis='x', labelsize=14)
        ax1.legend(loc='center left', fontsize=12)
        ax1.grid(True)
        
        ## Flux Plot 
            
        ax2 = ax1.twinx()

        ax2.plot(df['x'], df['diffFluxU2_x'],'b*-',label=flux_basis_functions_type)
        
        ax2.set_ylabel(r"$q_h(x)$ [mole/ft2-hr]", fontsize=16, color='lightblue')
        ax2.tick_params(axis='y', labelcolor='lightblue', labelsize=14)
        ax2.legend(loc='center right', fontsize=12)
        
        
    if (axial_2 == dimension):
        ax1.plot(df['x'], df['u'],'r*-',label=basis_functions_type)

        ax1.set_xlabel(r'$x$ [ft]', fontsize=18)
        ax1.set_ylabel(r'$u_h(x)$ [mole/ft3]', fontsize=18, color='red')

        ax1.tick_params(axis='y', labelcolor='red', labelsize=14)
        ax1.tick_params(axis='x', labelsize=14)
        ax1.legend(loc='center left', fontsize=12)
        ax1.grid(True)
        
        ## Flux Plot 
            
        ax2 = ax1.twinx()

        ax2.plot(df['x'], df['u2'],'b*-',label=flux_basis_functions_type)
        
        ax2.set_ylabel(r"$q_h(x)$ [mole/ft2-hr]", fontsize=16, color='lightblue')
        ax2.tick_params(axis='y', labelcolor='lightblue', labelsize=14)
        ax2.legend(loc='center right', fontsize=12)    
        
    
    plt.title(title, fontsize=20)
  
    plt.show()
    print('') 
    

In [ ]:
'''Domain'''

x_a = 0  #ft
x_b = 6  #ft

x_length = x_b - x_a

In [ ]:

'''Parameters and data'''

diff_coeff_1 = 0.00005 #ft^2/hr
diff_coeff_2 = 0.000092 #ft^2/hr
Mass_transfer_coeff = 2 #ft/hr
vel = [14735.8 ,0 ,0]  # ft/hr
Void_fraction = 0.002 
Interfacial_area= 7.5 #1/ft
Henry_constant= 2.08e-4 
mesh_type = 'edge3'

u_a = 0.0459   #mole/ft^3
u_b = 0        #mole/ft2-hr
u2_a = 0       #mole/ft^3
u2_b = 0       #mole/ft2-hr

'''FEM Solution'''

n_felem = 20

order = 'second'

n_plot_pts = n_felem + 1
    
from tools.toolkit  import write_engy5310_p2_1d_input_file  


write_engy5310_p2_1d_input_file(xmin=x_a, 
                                xmax=x_b, 
                                u_left=u_a,
                                u_right=u_b, 
                                u2_left=u2_a, 
                                u2_right=u2_b, 
                                diff_coeff_1 = diff_coeff,
                                diff_coeff_2 = diff_coeff_2 ,
                                velocity=velocity, 
                                n_felem=n_felem, 
                                order=order,
                                mesh_type = elem_type,
                                n_plot_pts=n_plot_pts,
                                compute_diffusion_flux=True,
                                use_moose_neumann_bc = True,
                                
                                solver='fdp-newt-full')

'''Display MOOSE input file created'''

!cat engy5310p2/input.hit

'''Run Engy5310P1 MOOSE App'''

!engy5310p2/engy5310p2-opt -i engy5310p2/input.hit

## [Results and Analysis](#toc)<a id="ranalysis"></a>

- From the graph we can see the concentration of xenon in salt is decreased by a small amount. As the system is dominated by convection, the diffusion has no effect on xenon transfer. Same amount of xenon concentration is increased in helium.Actually here I assumed a constant interfacial area and void. But both should increase from inlet to exit of pipe. For the simplification of model both terms were considered constant.


In [ ]:

'''Show FEM Solution'''

import pandas as pd
df = pd.read_csv('output_x-data_0002.csv')
    
plot_solution(df, dimension='x', title='FEM Solution of Xenon in Salt', basis_functions_type='u-Second Lagrange', flux_basis_functions_type='q-First Monomial')

In [ ]:

'''Show FEM Solution'''

import pandas as pd
df = pd.read_csv('output_x-data_0002.csv')
    
plot_solution(df, dimension='x1', title='FEM Solution of Xenon in Helium ', basis_functions_type='u-Second Lagrange', flux_basis_functions_type='q-First Monomial')

In [ ]:
'''Show FEM Solution'''

import pandas as pd
df = pd.read_excel (r'https://github.com/dpploy/engy-5310/blob/anadi/projects/xenon-removal/matlab.xlsx', sheet_name='matlab')
#df = pd.read_excel('matlab.xlsx')
    
plot_solution(df, dimension='x2', title='FEM Solution of Xenon in Helium ', basis_functions_type='u-Second Lagrange', flux_basis_functions_type='q-First Monomial')

## [Conclusions](#toc)<a id="conclusions"></a>

- The concentration of xenon in fuel salt is decreased due to transfer of xenon from salt to helium bubble. MOOSE and analytical solution gave the same result. To improve the result I need to consider several variable term in the governing equation. This is the simplest forms and considerations to simulate the concentration change of xenon in molten salt reactor.

## [References](#toc)<a id="refs"></a>

- [1] V. F. de Almeida, [*Engy-5310: Computational Continuum Transport Phenomena*](https://github.com/dpploy/engy-5310), University of Massachusetts Lowell, Dept. of Chemical Engineering (Nuclear Energy Program).
- [2] Multiphysics Object-Oriented Simulation Environment [(MOOSE)](https://mooseframework.org)
- [3] author, [reference]()
- [4] author, [reference]()